<a href="https://colab.research.google.com/github/kimgeonhee317/stable-diffusion-lec-notes/blob/main/notebook/lec_note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Stable diffusion basics

### Types of Image generation
1. Unconditional image generation: model just generates images without any additional condition like text and image. You will get images similar to provided in the training set.

2. Based on text : text to image, text2img
The prompt is converted to embedding

3. Based on image : image to image, img2img

### Diffusion Model
1. Diffusion models are essentially Markov chains trained using variational inference.
2. The purpose of diffusion models is to learn the latent structure of a dataset by modeling the way data points diffuse through the latent space
3. In the field of computer vision, this means that a neural network is trained to reduce noise from images blurred with Gaussian noise by learning how to reverse the diffusion process.
4. In other words, it is the process that will transform noise (the initial state) into generated image (the result)

### Autoencoders
Intorduced as a powerful tool to compress images or data in general.
- Encoder: converts the input into lower-dimensional latent vector, usually through standard convolutions and clustering layers.
- Decoder: performs operations such as deconvolution and upsampling to try to reconstruct the input image

### Variational autoencoders
+ The encoder returns the mean and the standard deviation for each input; then, the latent vector is sampled from this distribution and sent to the decoder to reconstruct the input
+ Training is now controlled by reconstruction loss and similarity loss, which is the KL divergence between the latent space distribution and the standard Gaussian.
+ VAEs are trained not only to reconstruct images, but also to produce latent vectors from a normal distribution

### U-NET
+ encoder: the first part and it is usually a pre-trained classification network (like VGG/ResNet).
Convolution blocks followed by maxpool/downsampling are applied to encode the input image into feature representations at several different levels

+ decoder: it is the second part of the architecture, which has the goal to semantically project the discriminative features (lower resolution) learned by the encoder into pixel space (higher resolution) to obtain a dense classification. The decoder consists of upsampling and concatenation followed by regular convolution operations.

### Text-encoder
+ The text-encoder is responsible for turning the input prompt into a embedding space that can be understood by U-net.
+ It is usally a simple transformer=based encoder that maps a squence of input tokens to a sequence of latent text-embeddings.
+ Inspired by Imagen, Stable diffusion does not train the text encoder during training, it simply uses an already trained CLIP text encodert, the **CLIPTextModel**

### Stable diffusion inference
+ SD receives a latent seed and a text prompt as input
+ The seed is used to generate random represntations of latent images of size 64 * 64 (4096)
+ The text prompt is transformed into text embeddings of size 77 * 768 using CLIP text encoder. (Maximum sequence word numbers is 77)
+ U-net iteratively reduces noise from the random latent image representations while conditioning on the text embeddings. => Text conditioned latent Unet
+ The U-Net output(noise residual) is used to compute a denoised latent image representation using a scheduler algorithm.
+ The denoising process is prepeated x times to recover the best latent image representations.
+ Once completed, the latent image representation is decoded by the decoder part of the VAE



### Recap Components
+ text encoder: Stable diffusion uses CLIP, but other diffusion models can use other encoders like BERT.
+ tokenizer: It must match the one used by the text_encoder model.
+ scheduler: The scheduler algorithm used to progressively add noise to the image during training
+ u-net: The model used to generate the latent representation of the input.
+ vae: Autoencoder module that we will use to decode latentrepresentations into real images.


### Scheduler algorithms
+ Scheduler algorithms (also called samplers) calculates the predicted denosied image representation from the previous noise representation and the predicted residual noise.
+ Determines how the image is calculated.
+ There are several different algorithms. Some examples commonly used with stable diffusion.
  + PNDM (default)
  + DDIM scheduler
  + K-LMS scheduler
  + Euler Ancestral Discrete Scheduler
  + DPM Scheduler

